In [5]:
from caffe2.python import workspace
import numpy as np

class CaffeClassifier(object): 
    
    def __init__(self):
       
        with open('/microservice/init_net.pb') as f:
            init_net = f.read()
        with open('/microservice/predict_net.pb') as f:
            predict_net = f.read()

        self.model = workspace.Predictor(init_net, predict_net)

 
    def predict(self,X,feature_names):
        return self.model.run({'data': X})

ModuleNotFoundError: No module named 'caffe2'

In [3]:
!s2i build . seldonio/seldon-core-s2i-python3 caffe-classifier:0.1

---> Installing application source...
---> Installing dependencies ...
You must give at least one requirement to install (see "pip help install")
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [8]:
!docker run --name "caffe_predictor" -d --rm -p 5000:5000 tejdeep9/caffeclassifier:0.1

38f0363b05e07a3263fc8c7637ae2d3a7d11f89bf03d1385cc0e50988bc3b75b


In [4]:
!cd ../../../../seldon-core/wrappers/testing && make build_protos

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


In [5]:
!python ../../../../seldon-core/wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:
{'meta': {}, 'data': {'names': ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x60', 'x61', 'x62', 'x63', 'x64', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70', 'x71', 'x72', 'x73', 'x74', 'x75', 'x76', 'x77', 'x78', 'x79', 'x80', 'x81', 'x82', 'x83', 'x84', 'x85', 'x86', 'x87', 'x88', 'x89', 'x90', 'x91', 'x92', 'x93', 'x94', 'x95', 'x96', 'x97', 'x98', 'x99', 'x100', 'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x107', 'x108', 'x109', 'x110', 'x111', 'x112', 'x113', 'x114', 'x115', 'x116', 'x117', 'x118', 'x119', 'x120', 'x121', 'x122', 'x123', 'x124', 'x125', 'x126', 'x127'

Traceback (most recent call last):
  File "/Users/tejdeepkautharam/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/Users/tejdeepkautharam/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/Users/tejdeepkautharam/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/tejdeepkautharam/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/Users/tejdeepkautharam/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 357, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/Users/tejdeepkau

In [49]:
!docker rm caffe_predictor --force


Error: No such container: caffe_predictor


In [50]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

Starting local Kubernetes v1.9.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


In [51]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default


clusterrolebinding.rbac.authorization.k8s.io "kube-system-cluster-admin" created


In [52]:
!helm init

$HELM_HOME has been configured at /Users/tejdeepkautharam/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [58]:
!helm install ../../../../seldon-core/helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../../seldon-core/helm-charts/seldon-core --name seldon-core

NAME:   seldon-core-crd
LAST DEPLOYED: Mon Jul  2 16:28:08 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s


NOTES:
NOTES: TODO


NAME:   seldon-core
LAST DEPLOYED: Mon Jul  2 16:28:08 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        0s

==> v1/RoleBindi

In [59]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3 caffe-classifier:0.1

---> Installing application source...
---> Installing dependencies ...
You must give at least one requirement to install (see "pip help install")
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [60]:
!kubectl create -f deploy.json

seldondeployment.machinelearning.seldon.io "caffe-test" created


In [69]:
!kubectl get seldondeployments caffe-test -o jsonpath='{.status}'


map[predictorStatus:[map[name:caffe-test-single-model replicas:1 replicasAvailable:0]]]

In [23]:
!cd ../../../../seldon-core/util/api_tester && make build_protos

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
mkdir -p ./proto
touch ./proto/__init__.py
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


In [71]:
!python ../../../../seldon-core/util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

----------------------------------------
SENDING NEW REQUEST:
{'meta': {}, 'data': {'names': ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x60', 'x61', 'x62', 'x63', 'x64', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70', 'x71', 'x72', 'x73', 'x74', 'x75', 'x76', 'x77', 'x78', 'x79', 'x80', 'x81', 'x82', 'x83', 'x84', 'x85', 'x86', 'x87', 'x88', 'x89', 'x90', 'x91', 'x92', 'x93', 'x94', 'x95', 'x96', 'x97', 'x98', 'x99', 'x100', 'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x107', 'x108', 'x109', 'x110', 'x111', 'x112', 'x113', 'x114', 'x115', 'x116', 'x117', 'x118', 'x119', 'x120', 'x121', 'x122', 'x123', 'x124', 'x125', 'x126', 'x127'

{"access_token":"25a0c013-b13e-4ba4-ac93-bbef31dea622","token_type":"bearer","expires_in":43122,"scope":"read write"}
RECEIVED RESPONSE:
{'code': 103, 'info': 'org.apache.http.conn.ConnectTimeoutException: Connect to caffe-test:8000 [caffe-test/10.111.208.227] failed: connect timed out', 'reason': 'Microservice error', 'status': 'FAILURE'}



In [36]:
!minikube delete

Deleting local Kubernetes cluster...
Machine deleted.
